<a href="https://colab.research.google.com/github/CorentinGaillard/GTNH_craft_calc/blob/main/GTNH_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GTNH Calculator

It takes in a csv with all the recipe you are interested in, and outputs a list of all the ressources you will need

In [42]:
import pandas as pd, numpy as np, requests
from collections import defaultdict
from json import dumps

In [43]:
# reading the csv file with the recipes from github:
file = "https://raw.githubusercontent.com/CorentinGaillard/GTNH_craft_calc/main/GTNH%20circuits%20-%20recipe.csv"
df = pd.read_csv(file)

In [44]:
# Filling in NaNs
tmp = df.drop(columns=['Byproduct', 'quantity.2']).fillna(method='ffill')
tmp['Byproduct'] = df.Byproduct
tmp['quantity.2'] = df['quantity.2']
df = tmp
del tmp
df

,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2
0,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Good Intergrated Circuit,1,NaN,NaN
1,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Integrated Logic Circuit,2,NaN,NaN
2,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Random Access Memory Chip,2,NaN,NaN
3,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,SMD Transistor,4,NaN,NaN
4,Advanced Circuit,1.0,30.0,24000.0,800.0,40.00,Circuit assembler,Fine electrum Wire,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
75,Copper Foil,4.0,24.0,1512.0,63.0,3.15,Bending Machine,Copper Plate,1,NaN,NaN
76,Copper Foil,4.0,24.0,1512.0,63.0,3.15,Bending Machine,Programmed Cuircuit (1) (Not consumed),1,NaN,NaN
77,Copper Plate,1.0,24.0,1512.0,63.0,3.15,Bending Machine,Copper Ingot,1,NaN,NaN
78,Copper Plate,1.0,24.0,1512.0,63.0,3.15,Bending Machine,Programmed Cuircuit (1) (Not consumed),1,NaN,NaN


In [45]:
def get(nb_item, name_item, df, mult=1):
  global ingredients, intermediates
  # If no recipe, return the name and the quantity
  if not name_item in df.obj.values:
    return  (nb_item*mult, name_item)
  # else, get the ingredients for the subcraft, and call the function on each ingredient
  componants = df.loc[df.obj==name_item, ["ressources in", "quantity.1"]]
  for name, nb in componants.values:
    qn = get(nb, name, df, mult=nb_item*mult/df.loc[df.obj==name_item, "quantity"].values[0])
    if qn:
    # if (qn := get(nb, name, df, mult=nb_item*mult/df.loc[df.obj==name_item, "quantity"].values[0])): # Python 3.9 is not here yet TT
      # if the function return, it means that there is no recipe for this craft, thus add it to the ingredients dict
      ingredients[qn[1]] += qn[0] #* mult
  intermediates[name_item] += nb_item*mult

In [46]:
def get_quantity_and_recipe(name, nb_item):
  global ingredients, intermediates
  ingredients = defaultdict(int)
  intermediates = defaultdict(int)
  get(nb_item, name, df)
  def boolify(d):
    for name in d.keys():
      if '(Not consumed)' in name:
        d[name] = int(bool(d[name]))
  boolify(ingredients)
  boolify(intermediates)
  idx = df.loc[df.obj==intermediates.keys()].index
  for name, nb in intermediates.items():
    idx = np.r_[idx, df.loc[df.obj==name].index]
    mask = df.obj==name
    df.loc[mask, "quantity to craft"] = nb
    df.loc[mask, "total eu craft"] = df.loc[mask, "total eu"] * nb
    df.loc[mask, "total seconds craft"] = df.loc[mask, "total seconds"] * nb
  return df.loc[idx]

In [47]:
# The available recipe are:
df.obj.unique()

array(['Advanced Circuit', 'Good Intergrated Circuit',
       'Integrated Logic Circuit', 'SMD Transistor', 'SMD Resistor',
       'Diode', 'Random Access Memory Chip',
       'Integrated Logic Circuit Chip', 'Good Circuit board',
       'Circuit Board', 'Random Access Memory Chip (Wafer)',
       'Integrated Logic Circuit (Wafer)', 'Phenolic Circuit Board',
       'Fine electrum Wire', 'Electrum Wire', 'Annealed Copper Bolt',
       'Silver Bolt', 'Fine Copper Wire', 'Copper Wire', 'tin bolt',
       'Gallium Foil', 'Gallium Plate', 'Fine Annealed copper wire',
       'Annealed Copper Wire', 'Gold Foil', 'Gold Plate',
       'Iron III Chloride', 'Hydrocholric Acid', 'Chlorine', 'Wood Plank',
       'Copper Foil', 'Copper Plate', 'Wood Pulp'], dtype=object)

In [48]:
# to get the componant for a recipe, you call the function, 
# then you look into df and ingredients
out = get_quantity_and_recipe("Advanced Circuit", 64)
print(dumps(ingredients, indent=2))
out

{
  "Log": 181.33333333333331,
  "Refined Glue": 2304.0,
  "Programmed Cuircuit (1) (Not consumed)": 1,
  "Gold Ingot": 64.0,
  "Iron Dust": 6.4,
  "Empty Cell ": 19.200000000000003,
  "Hydrogen Cell": 19.2,
  "Rock Salt": 38.4,
  "Copper Ingot": 128.0,
  "Molten Polyethylene": 41472.0,
  "Wafer": 228.0,
  "Ruby Lens (Not consumed)": 1,
  "Water": 6840.0,
  "SMD resistor": 512.0,
  "Annealed Copper Ingot": 176.0,
  "Fine Copper wire": 512.0,
  "Tin Ingot": 64.0,
  "Extruder Shape (Bolt) (Not consumed)": 1,
  "molten tin": 55296.0,
  "Carbon Dust": 16.0,
  "Electrum Ingot": 104.0,
  "Silver Ingot": 32.0,
  "Green Sapphire Lens (Not consumed)": 1,
  "Gallium Ingot": 4.0
}


,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2,quantity to craft,total eu craft,total seconds craft
79,Wood Pulp,6.0,2.0,800.0,400.0,20.0,Pulverization,Log,1,NaN,NaN,1088.0,870400.0,21760.0
44,Phenolic Circuit Board,1.0,16.0,4800.0,300.0,15.0,Assembler,Wood Pulp,1,NaN,NaN,64.0,307200.0,960.0
45,Phenolic Circuit Board,1.0,16.0,4800.0,300.0,15.0,Assembler,Refined Glue,36,NaN,NaN,64.0,307200.0,960.0
46,Phenolic Circuit Board,1.0,16.0,4800.0,300.0,15.0,Assembler,Programmed Cuircuit (1) (Not consumed),1,NaN,NaN,64.0,307200.0,960.0
65,Gold Plate,1.0,24.0,4704.0,196.0,9.8,Bending Machine,Gold Ingot,1,NaN,NaN,64.0,301056.0,627.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,Advanced Circuit,1.0,30.0,24000.0,800.0,40.0,Circuit assembler,Random Access Memory Chip,2,NaN,NaN,64.0,1536000.0,2560.0
3,Advanced Circuit,1.0,30.0,24000.0,800.0,40.0,Circuit assembler,SMD Transistor,4,NaN,NaN,64.0,1536000.0,2560.0
4,Advanced Circuit,1.0,30.0,24000.0,800.0,40.0,Circuit assembler,Fine electrum Wire,8,NaN,NaN,64.0,1536000.0,2560.0
5,Advanced Circuit,1.0,30.0,24000.0,800.0,40.0,Circuit assembler,Annealed Copper Bolt,8,NaN,NaN,64.0,1536000.0,2560.0


In [49]:
# tests
out = get_quantity_and_recipe("Wood Plank", 64)
print(dumps(ingredients, indent=2))
out
# 1 log = 6 pulp, 8 pulp = 1 plank -> 1 log = 6/8 = 0.75 plank
# 1 plank = 8/6 = 1.333... log
# 64 plank = 64* 8/6 = 85.3333 log OK

{
  "Log": 85.33333333333333
}


,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2,quantity to craft,total eu craft,total seconds craft
79,Wood Pulp,6.0,2.0,800.0,400.0,20.0,Pulverization,Log,1,NaN,NaN,512.0,409600.0,10240.0
74,Wood Plank,1.0,2.0,600.0,300.0,15.0,Compressor,Wood Pulp,8,NaN,NaN,64.0,38400.0,960.0


In [50]:
# tests
out = get_quantity_and_recipe("Phenolic Circuit Board", 64)
print(dumps(ingredients, indent=2))
out
# 1 board = 36 glue
# 64 board = 36*64 = 2304 glue OK

{
  "Log": 10.666666666666666,
  "Refined Glue": 2304.0,
  "Programmed Cuircuit (1) (Not consumed)": 1
}


,obj,quantity,eu/t,total eu,total ticks,total seconds,machine name,ressources in,quantity.1,Byproduct,quantity.2,quantity to craft,total eu craft,total seconds craft
79,Wood Pulp,6.0,2.0,800.0,400.0,20.0,Pulverization,Log,1,NaN,NaN,64.0,51200.0,1280.0
44,Phenolic Circuit Board,1.0,16.0,4800.0,300.0,15.0,Assembler,Wood Pulp,1,NaN,NaN,64.0,307200.0,960.0
45,Phenolic Circuit Board,1.0,16.0,4800.0,300.0,15.0,Assembler,Refined Glue,36,NaN,NaN,64.0,307200.0,960.0
46,Phenolic Circuit Board,1.0,16.0,4800.0,300.0,15.0,Assembler,Programmed Cuircuit (1) (Not consumed),1,NaN,NaN,64.0,307200.0,960.0
